In [ ]:
#! /usr/bin/env python

from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local[2]"))

import re

In [ ]:
n = 150
start, end = 12, 34

def parse_edge(s):
    user, follower = s.split("\t")
    return (int(user), int(follower))

def step(item):
    prev_v, prev_d, next_v = item[0], item[1][0], item[1][1]
    return (next_v, prev_d + [next_v])

edges = sc.textFile("/data/twitter/twitter_sample_small.txt").map(parse_edge).cache()

In [ ]:
forward_edges = edges.map(lambda e: (e[1], e[0])).partitionBy(n).persist()
paths = sc.parallelize([(start, [start])]).partitionBy(n)

def found():
    return  paths.filter(lambda x: x[0] == end).count()
while not found():
    paths = paths.join(forward_edges, n).map(step)
paths.cache()
final_paths = (paths
               .filter(lambda value: value[0] == end)
               .map(lambda value: value[1])
              ).cache()
result = final_paths.take(1)[0]
print(','.join(map(str,result)))